In [1]:
import random
import numpy as np
import tensorflow as tf

seed_value= 1111
random.seed(seed_value)
np.random.seed(seed_value)
tf.set_random_seed(seed_value)

import warnings
import sys
import os
import pandas as pd
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
# from sklearn.metrics import mean_absolute_percentage_error
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential,Model
from keras.layers import Input,LSTM, Dense, Flatten, Conv1D, Lambda, Reshape, RepeatVector
from keras.layers.merge import concatenate, multiply,add
from keras import regularizers
from keras.initializers import glorot_uniform
from tqdm import tqdm
from keras import regularizers
from keras.models import load_model
import datetime
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import random as python_random
from statsmodels.tsa.stattools import pacf
from matplotlib import pyplot

/home/chatta/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/chatta/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/chatta/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/chatta/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dty

In [8]:
# utility function to make input window for training and test sets

def make_input(data,window_size,horizon=1):
    length=data.shape[0]
    y = np.zeros([length-window_size+1-horizon,horizon])
    output=np.zeros([length-window_size+1-horizon,window_size])
    for i in range(length-window_size-horizon+1):
        output[i:i+1,:]=data[i:i+window_size]
        y[i,:]= data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],window_size), y

def make_k_input(data,horizon):
    length = data.shape[0]
    output= np.zeros([length+1-horizon,horizon])
    for i in range(length-horizon+1):
        output[i:i+1,:]=data[i:i+horizon]
    return output.reshape(output.shape[0],horizon)

def nonov_make_input(data,window_size,horizon=1):
    length=data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon

    data = np.append(data,np.zeros([horizon-extra]))

    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
        
    output=np.zeros([i_val,window_size])
    y=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[i*horizon:(i*horizon)+window_size]
        y[i,:]= data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
        
    return output.reshape(output.shape[0],window_size), y

def nonov_make_k_input(data,horizon):
    length = data.shape[0]
    loop=length//horizon
    extra = length%horizon
    data_app = np.repeat(data[-1],(horizon-extra))
    data = np.append(data,data_app)    

    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
    output=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[(i*horizon):(i*horizon)+horizon]
    return output.reshape(output.shape[0],horizon)


def metrics(pred,gt):
    l = pred.shape[1]
#     print(l)
    err_mse = np.zeros((l))
    err_mae = np.zeros((l))

    for i in range(l):
        err_mse[i] = mse(pred[:,i],gt[:,i])
        err_mae[i] = mae(pred[:,i],gt[:,i])
        
    return np.sqrt(np.mean(err_mse)),np.mean(err_mae)

def concat(data):
    temp=np.zeros((data.shape[0]*207,data.shape[1]))
    for i in range(207):
        temp[i*data.shape[0]:(i+1)*data.shape[0],:]=data[:,:,i]
    return temp

def window_normalize(data_x,data_y):
    
    min_in = data_x.min(1).reshape(data_x.shape[0],1)
    max_in = data_x.max(1).reshape(data_x.shape[0],1)
    denom = (max_in-min_in)
    a = np.where(denom == 0)[0]
    denom[a] = max_in[a] 
    a = np.where(denom == 0)[0]
    if a.size >0:
        denom[a]=1
    out = (data_x-min_in)/denom
    out=out.reshape(out.shape[0],out.shape[1],1)
    out_y=(data_y-min_in)/denom
    return out,out_y,denom,min_in

def std_window_normalize(data_x,data_y):
    
    mean_in = np.mean(data_x,axis=1).reshape(data_x.shape[0],1)
    std_in = np.std(data_x,axis=1).reshape(data_x.shape[0],1)
    
    a = np.where(std_in == 0)[0]
    std_in[a] = 1
    
    out = (data_x-mean_in)/std_in
    out=out.reshape(out.shape[0],out.shape[1],1)
    out_y=(data_y-mean_in)/std_in
    return out,out_y,std_in,mean_in

def p_window_normalize(data_x):
    
    min_in = data_x.min(1).reshape(data_x.shape[0],1)
    max_in = data_x.max(1).reshape(data_x.shape[0],1)
    denom = (max_in-min_in)
    a = np.where(denom == 0)[0]
    denom[a] = max_in[a] 
    a = np.where(denom == 0)[0]
    if a.size >0:
        denom[a]=1
    out = (data_x-min_in)/denom
    out=out.reshape(out.shape[0],out.shape[1])
    
    return out

def p_std_window_normalize(data_x):
    
    mean_in = np.mean(data_x,axis=1).reshape(data_x.shape[0],1)
    std_in = np.std(data_x,axis=1).reshape(data_x.shape[0],1)
    
    a = np.where(std_in == 0)[0]
    std_in[a] = 1
    
    out = (data_x-mean_in)/std_in
 
    out=out.reshape(out.shape[0],out.shape[1])
    
    return out

In [98]:

data = pd.read_csv('/home/chatta/logic_rules/data/traffic_30_with_std.csv',usecols=['avg_flow','std'])
std_data=np.asarray(data['std'])
data= np.asarray(data['avg_flow'])
data1=data[47:]
horizon = 1
window_size=3
knowledge_preds=np.asarray(pd.read_csv("/home/chatta/logic_rules/results/pacf_preds/traffic_30_std/h1/preds_noisy.csv",header=None))
n_train=int(0.8*data.size)+1
n_test=int(.1*data.size)
n_val=2*n_test



In [99]:


train_d = data1[:-n_val]
# n_train=train_d.size
# n_reduced = int(0.1*n_train)
# train_d=train_d[-(n_reduced+window_size):]
val_d=data1[-(n_val+window_size):-n_test]
test_d=data1[-(n_test+window_size):]

current_pred= knowledge_preds
train_p = current_pred[:-n_val]
# train_p=train_p[-n_reduced:]
val_p = current_pred[-n_val:-n_test]
test_p = current_pred[-n_test:]


1354

In [100]:

#--------------------------concept-----------------------------


train_sequence = make_input(train_d, window_size,horizon)
val_sequence = make_input(val_d,window_size,horizon)
test_sequence = nonov_make_input(test_d,window_size,horizon)



train_pred = make_k_input(train_p,horizon)
val_pred = make_k_input(val_p,horizon)
test_pred = nonov_make_k_input(test_p,horizon)

norm_train_x,norm_train_y,denom_train,min_train=std_window_normalize(train_sequence[0],train_sequence[1])
norm_val_x,norm_val_y,denom_val,min_val=std_window_normalize(val_sequence[0],val_sequence[1])  
norm_test_x,norm_test_y,denom_test,min_test=std_window_normalize(test_sequence[0],test_sequence[1])

norm_p_train_x=(train_pred-min_train)/denom_train 
norm_p_val_x=(val_pred-min_val)/denom_val
norm_p_test_x=(test_pred-min_test)/denom_test 

train_input=np.repeat(norm_p_train_x,window_size,axis=1)
val_input=np.repeat(norm_p_val_x,window_size,axis=1)
test_input=np.repeat(norm_p_test_x,window_size,axis=1)

train_input=train_input.reshape(train_input.shape[0],train_input.shape[1],1)
val_input=val_input.reshape(val_input.shape[0],val_input.shape[1],1)
test_input=test_input.reshape(test_input.shape[0],test_input.shape[1],1)

final_in_train =np.append(norm_train_x,train_input,axis=2)


final_in_val =np.append(norm_val_x,val_input,axis=2)


final_in_test =np.append(norm_test_x,test_input,axis=2)

final_lbl_train = norm_train_y
final_lbl_val= norm_val_y
final_lbl_test = norm_test_y

final_p_train= norm_p_train_x
final_p_test= norm_p_test_x
final_p_val= norm_p_val_x




In [91]:
train_input.shape

(10785, 3, 1)

In [101]:
tf.reset_default_graph()
K.clear_session()  





input_data= Input(batch_shape=(None,window_size,2),name='input_data')
input_pred=Input(batch_shape=(None,horizon),name='input_pred')


# branch_0 = Conv1D(4,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform())(input_data)
# branch_1 = Conv1D(32,3, strides=1,dilation_rate=1,padding='same',activation='relu',kernel_initializer=glorot_uniform())(branch_0)
# branch_2 = Conv1D(64,3, strides=1,dilation_rate=1, padding='same',activation='relu',kernel_initializer=glorot_uniform())(branch_1)

# branch_4=Flatten()(branch_2)
# net= Dense(128,activation='relu')(branch_4)
# net= Dense(horizon,activation='relu')(net)
# net=add([net,input_pred])
#         ,activity_regularizer=regularizers.l2(0.01)

branch_0 = LSTM(512,activation='sigmoid',kernel_initializer=glorot_uniform(1),return_sequences=True,name='lstm_1')(input_data)
branch_1 = LSTM(64,activation='relu',kernel_initializer=glorot_uniform(2),return_sequences=True,name='lstm_2')(branch_0)
branch_2 = LSTM(16,activation='relu',kernel_initializer=glorot_uniform(3),name='lstm_3')(branch_1)
net= Dense(horizon,name='dense_final2')(branch_2)
# net=add([net,input_pred])

# model=Model(inputs=[input_data,input_pred],outputs=net)
model=Model(inputs=[input_data],outputs=net)

In [93]:
callback = ModelCheckpoint(filepath='/home/chatta/logic_rules/model_checkpoints/concept/final_weights/n100_1.h5',monitor='val_loss',save_best_only=True)

model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))
hist=model.fit({'input_data':final_in_train,'input_pred':final_p_train},final_lbl_train,validation_data=[[final_in_val,final_p_val],final_lbl_val],callbacks=[callback],batch_size=5*512,shuffle=False, epochs=300,verbose=1)

Train on 10785 samples, validate on 1354 samples
Epoch 1/300
10785/10785 [==============================] - 3s 255us/step - loss: 60.9611 - val_loss: 207.2730
Epoch 2/300
10785/10785 [==============================] - 0s 17us/step - loss: 60.3084 - val_loss: 206.5359
Epoch 3/300
10785/10785 [==============================] - 0s 16us/step - loss: 59.6171 - val_loss: 205.6870
Epoch 4/300
10785/10785 [==============================] - 0s 16us/step - loss: 58.7923 - val_loss: 204.6285
Epoch 5/300
10785/10785 [==============================] - 0s 16us/step - loss: 57.7402 - val_loss: 203.2504
Epoch 6/300
10785/10785 [==============================] - 0s 16us/step - loss: 56.3563 - val_loss: 201.4150
Epoch 7/300
10785/10785 [==============================] - 0s 15us/step - loss: 54.5548 - val_loss: 199.2045
Epoch 8/300
10785/10785 [==============================] - 0s 15us/step - loss: 52.5999 - val_loss: 197.2423
Epoch 9/300
10785/10785 [==============================] - 0s 15us/step - loss

10785/10785 [==============================] - 0s 15us/step - loss: 11.0773 - val_loss: 84.6567
Epoch 76/300
10785/10785 [==============================] - 0s 16us/step - loss: 10.9849 - val_loss: 84.5256
Epoch 77/300
10785/10785 [==============================] - 0s 16us/step - loss: 10.9314 - val_loss: 83.8136
Epoch 78/300
10785/10785 [==============================] - 0s 16us/step - loss: 10.8537 - val_loss: 83.2275
Epoch 79/300
10785/10785 [==============================] - 0s 16us/step - loss: 10.7730 - val_loss: 82.5900
Epoch 80/300
10785/10785 [==============================] - 0s 16us/step - loss: 10.7355 - val_loss: 82.0911
Epoch 81/300
10785/10785 [==============================] - 0s 15us/step - loss: 10.6396 - val_loss: 81.6924
Epoch 82/300
10785/10785 [==============================] - 0s 15us/step - loss: 10.5748 - val_loss: 81.1485
Epoch 83/300
10785/10785 [==============================] - 0s 15us/step - loss: 10.5381 - val_loss: 80.6752
Epoch 84/300
10785/10785 [======

10785/10785 [==============================] - 0s 15us/step - loss: 8.3760 - val_loss: 55.8061
Epoch 151/300
10785/10785 [==============================] - 0s 15us/step - loss: 8.3569 - val_loss: 55.6762
Epoch 152/300
10785/10785 [==============================] - 0s 15us/step - loss: 8.3372 - val_loss: 55.4700
Epoch 153/300
10785/10785 [==============================] - 0s 15us/step - loss: 8.3450 - val_loss: 55.1892
Epoch 154/300
10785/10785 [==============================] - 0s 15us/step - loss: 8.2431 - val_loss: 54.8854
Epoch 155/300
10785/10785 [==============================] - 0s 15us/step - loss: 8.2448 - val_loss: 54.7892
Epoch 156/300
10785/10785 [==============================] - 0s 15us/step - loss: 8.2690 - val_loss: 54.4760
Epoch 157/300
10785/10785 [==============================] - 0s 15us/step - loss: 8.2054 - val_loss: 54.2030
Epoch 158/300
10785/10785 [==============================] - 0s 15us/step - loss: 8.2149 - val_loss: 54.0763
Epoch 159/300
10785/10785 [======

10785/10785 [==============================] - 0s 15us/step - loss: 7.5956 - val_loss: 42.8927
Epoch 226/300
10785/10785 [==============================] - 0s 15us/step - loss: 7.6289 - val_loss: 42.8621
Epoch 227/300
10785/10785 [==============================] - 0s 15us/step - loss: 7.5849 - val_loss: 42.6096
Epoch 228/300
10785/10785 [==============================] - 0s 15us/step - loss: 7.6084 - val_loss: 42.5337
Epoch 229/300
10785/10785 [==============================] - 0s 15us/step - loss: 7.5755 - val_loss: 42.3657
Epoch 230/300
10785/10785 [==============================] - 0s 16us/step - loss: 7.6060 - val_loss: 42.2972
Epoch 231/300
10785/10785 [==============================] - 0s 15us/step - loss: 7.5615 - val_loss: 42.0362
Epoch 232/300
10785/10785 [==============================] - 0s 15us/step - loss: 7.6003 - val_loss: 41.9983
Epoch 233/300
10785/10785 [==============================] - 0s 15us/step - loss: 7.5574 - val_loss: 41.8162
Epoch 234/300
10785/10785 [======

10785/10785 [==============================] - 0s 16us/step - loss: 7.3676 - val_loss: 34.8392


In [106]:
#-------------------correponding weights are---------------
# for case 1=100_1, case= 2=50_1, case:3 =10_1 case4: n10_1

model=load_model(path+'logic_rules/model_checkpoints/concept/final_weights/100_1.h5')

In [107]:
preds=model.predict({'input_data':final_in_test,'input_pred':final_p_test})*denom_test+min_test


mse(preds,data[-n_test:])

7.380929522254613

In [108]:
np.savetxt('/home/chatta/logic_rules/model_checkpoints/results/simple100.txt',preds, fmt='%1.5f',delimiter=',')